In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.metrics import classification_report,accuracy_score
from sklearn import tree

In [ ]:
# Read the data into a dataframe
df = pd.read_csv("../input/csgo-round-winner-classification/csgo_round_snapshots.csv")
df

In [ ]:
# Check for null columns
df.isnull().sum()

In [ ]:
# Plot round winners by sides
sns.countplot(df['round_winner'])

In [ ]:
col = df.columns
print(col)

In [ ]:
# Remove columns with only 1 value
val_count = []
for i in col:
    val_count.append(df[i].nunique())

removed = []

for i in range(len(val_count)):
    if val_count[i] == 1:
        removed.append(col[i])
        df.drop(col[i], axis = 1, inplace = True)


In [ ]:
# New df with less columns
df

In [ ]:
# Removed columns
removed

In [ ]:
# View value counts in columns
val_count

In [ ]:
# Label encoding categorical values
le = preprocessing.LabelEncoder()
df["map"] = le.fit_transform(df["map"])
df['bomb_planted'] = le.fit_transform(df['bomb_planted'])
df['round_winner'] = le.fit_transform(df['round_winner'])
df

In [ ]:
# Classifier with only 11 first features
X_11 = df[["time_left","ct_score","t_score","bomb_planted","map","ct_health","t_health","ct_armor",
                       "t_armor","ct_money","t_money"]]
X_11.head()

In [ ]:
y = df["round_winner"]
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_11, y, test_size = 0.33, random_state = 200)
X_train

In [ ]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
cr = classification_report(y_test,pred)
print(cr)

In [ ]:
# Classifier with all features
X = df.drop('round_winner', axis = 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 200)
X_train

In [ ]:
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)
pred = rfc.predict(X_test)
cr = classification_report(y_test,pred)
print(cr)